In [1]:
%pip install langchain-text-splitters transformers pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd

In [3]:
from transformers import AutoTokenizer
from langchain_text_splitters import CharacterTextSplitter

# Load a tokenizer for a BERT-like model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
def count_tokens(text):
    return len(tokenizer.encode(text))

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=400,
    chunk_overlap=50,
    # length_function=len,
    length_function=count_tokens,
    # is_separator_regex=False,
)

In [4]:
def chunk(text):
    texts = text_splitter.create_documents([text])
    chunks = []
    for i,t in enumerate(texts):
        # replace newlines with spaces this can help keep word boundires
        chunks.append(t.page_content.replace("\n", " "))
    return chunks

In [5]:
df = pd.read_csv('./datasets/text_classes.csv', delimiter='|')
df = df.sample(frac=1)
# 4353
# display(df)

In [6]:
from tables import classes, schema_dicts
print(classes)
print(schema_dicts)

['Intellectual Developmental Disorders', 'Communication Disorders', 'Autism Spectrum Disorder', 'Attention-Deficit/Hyperactivity Disorder', 'Specific Learning Disorder', 'Motor Disorders', 'Other Neurodevelo mental Disorders p', 'Schizophrenia Spectrum and Other Psychotic Disorders', 'Catatonia', 'Bipolar and Related Disorders', 'Depressive Disorders', 'Anxiety Disorders', 'Obsessive-Compulsive and Related Disorders', 'Trauma- and Stressor-Related Disorders', 'Dissociative Disorders', 'Somatic Symptom and Related Disorders', 'Feeding and Eating Disorders', 'Elimination Disorders', 'Sleep-Wake Disorders', 'Breathing-Related Sleep Disorders', 'Parasomnias', 'Sexual Dysfunctions', 'Gender Dysphoria', 'Disruptive, Impulse-Control, and Conduct Disorders', 'Neurocognitive Disorders', 'Personality Disorders', 'Cluster A Personality Disorders', 'Cluster B Personality Disorders', 'Cluster C Personalit Disorders y', 'Other Personality Disorders', 'Paraphilic Disorders', 'Other Mental Disorders a

In [7]:
schema_index = 3
max_avg_distance = 15

In [8]:
import weaviate
import weaviate.classes as wvc

def query_top_five(text, limit=1, max_distance=20):
    client = weaviate.connect_to_local()
    res = None
    try:
        collection = client.collections.use(schema_dicts[schema_index]["schema"])

        results = collection.query.near_text(
            # Because near_text is a purely vector search, we get a
            # distance and no score
            return_metadata=wvc.query.MetadataQuery(
                score=True, explain_score=True, distance=True, certainty=True
            ),
            query=text,
            limit=limit
        )
        for r in results.objects:
            if r.metadata.distance < max_distance:
                res = [classes.index(r.properties['title']), r.metadata.distance]

        return res
    finally:
        client.close()

In [9]:
# item = 1

# chart_chunks = chunk(df.iloc[item]["chart_text"])
# print(f"Chunks size: {len(chart_chunks)}")

# scores = []
# for c in chart_chunks:
#     d = query_top_five(c)
#     if d is not None:
#         scores.append(d)

# print(f"Scores size: {len(scores)}")

In [10]:
def score_document(s, max=10):
    cat = {}
    top_results = []

    for i, _ in enumerate(classes):
        cat[i] = {"count":0, "distance":0}

    for _, d in enumerate(s):
        cat[d[0]]["count"] += 1
        cat[d[0]]["distance"] += d[1]
        cat[d[0]]["avg"] = cat[d[0]]["distance"] / cat[d[0]]["count"]

    for i, _ in enumerate(classes):
        if cat[i]["count"] > 0 and cat[i]["avg"] < max:
            top_results.append([cat[i]["avg"], i])

    return top_results

# top = np.array(score_document(scores, max_avg_distance))
# print(np.sort(top, axis=0))
# print('--------------')
# print(f'{df.iloc[item]["class_number"]:02} {classes[df.iloc[item]["class_number"]]}')

In [11]:
test_range = 20

results = []
for i in range(test_range): # 4353//4):
    chart_chunks = chunk(df.iloc[i]["chart_text"])
    print(f"Chunks size: {len(chart_chunks)}")

    scores = []
    for c in chart_chunks:
        d = query_top_five(c)
        if d is not None:
            scores.append(d)

    top = np.array(score_document(scores, max_avg_distance))
    top = np.sort(top, axis=0)

    results.append([i, df.iloc[i]["class_number"], top])

Chunks size: 102
Chunks size: 23
Chunks size: 8
Chunks size: 4
Chunks size: 24
Chunks size: 6


Created a chunk of size 480, which is longer than the specified 400


Chunks size: 18
Chunks size: 48
Chunks size: 103
Chunks size: 10
Chunks size: 45
Chunks size: 5
Chunks size: 35


Created a chunk of size 439, which is longer than the specified 400


Chunks size: 29
Chunks size: 26


Created a chunk of size 434, which is longer than the specified 400


Chunks size: 21
Chunks size: 4
Chunks size: 5
Chunks size: 52
Chunks size: 949


In [12]:
df_results = pd.DataFrame(results, columns=["record", "original", "predict"])

def top5(x):
    return np.array([y[1] for i,y in enumerate(x) if i < 5])

def in_top_5(orig, top5):
    return 1 if orig in top5 else 0

df_results['top5'] = df_results['predict'].map(top5)
df_results['inTop5'] = df_results.apply(lambda x: in_top_5(orig=x['original'], top5=x['top5']), axis=1)

print(df_results['inTop5'].sum() / test_range)

display(df_results)

0.7


,record,original,predict,top5,inTop5
0,0,10,"[[7.878668573167589, 9.0], [8.774086475372314,...","[9.0, 10.0, 16.0, 17.0, 18.0]",1
1,1,10,"[[7.601770401000977, 9.0], [9.37295389175415, ...","[9.0, 10.0, 17.0, 18.0, 19.0]",1
2,2,10,"[[9.353336970011393, 10.0], [10.36445776621500...","[10.0, 23.0]",1
3,3,11,"[[7.409327983856201, 18.0], [8.699587345123291...","[18.0, 20.0, 33.0]",0
4,4,10,"[[7.114090919494629, 9.0], [7.589475750923157,...","[9.0, 16.0, 18.0, 19.0, 23.0]",0
5,5,10,"[[7.121258926391602, 10.0], [8.127206802368164...","[10.0, 23.0]",1
6,6,11,"[[8.195616006851196, 9.0], [8.31314468383789, ...","[9.0, 10.0, 16.0, 23.0, 33.0]",0
7,7,10,"[[7.679414351781209, 9.0], [8.058435440063477,...","[9.0, 10.0, 12.0, 17.0, 18.0]",1
8,8,10,"[[8.35844171408451, 10.0], [8.680580298105875,...","[10.0, 16.0, 18.0, 19.0, 20.0]",1
9,9,10,"[[6.885260581970215, 10.0], [8.75302004814148,...","[10.0, 18.0, 23.0, 24.0]",1
